Please complete the specified sections. You may modify the code as needed; this is intended only as a guide or template.

# Modules
run this block to download pysat module

In [90]:
!pip install python-sat

In [91]:
from pysat.formula import CNF
from pysat.solvers import Solver
import copy

# Atoms

In [92]:
#global Dictionaries for encoded atoms
atoms_dict = {} # (kind, i, j, t) => ID
reverse_atoms_dict ={} # ID => (kind, i, j, t)
lastUsedID = 0

Guide:

 1. Use binary (0, 1) for the "kind" input to indicate a selection between a player or a box:
* Use 1 to represent a player.
* Use 0 to represent a box.

2. Use the encodeAtom function to generate a positive number (code) for atoms. For example, to represent the existence of a player at location (i, j) on board "t", the encodeAtom call will be encodeAtom(1, i, j, t).


In [93]:
#Encode variables to a positive number
def encodeAtom(kind, i, j, t):
        global lastUsedID, atoms_dict, reverse_atoms_dict

        if not ((kind, i, j, t) in atoms_dict):
          lastUsedID += 1
          atoms_dict[(kind, i, j, t)] =  lastUsedID
          reverse_atoms_dict[lastUsedID] = (kind, i, j, t)
        return atoms_dict[(kind, i, j, t)]

At last you can decode ID to Atom properties which is a tuple of the form (kind, i, j, t).

In [94]:
#Decode Integers to Atoms
def decodeAtom(ID):
    global reverse_atoms_dict
    if abs(ID) in reverse_atoms_dict:
        return reverse_atoms_dict[abs(ID)]
    return -1

# Coditions

## 1. INITIAL BOARD
Generate clauses for the initial board by setting atoms based on the initial board configuration.

Adding Clause Guide:
- Use clauses.append([p1, p2, ..., p(n)]) to add a clause in CNF format, representing (p1 OR p2 OR ... OR p(n)).
- Each atom is represented by a positive integer (which is handled by encodeAtom/decodeAtom functions), and the negation of an atom is represented by the negative of that integer. Therefore, -1*p1 is equivalent to NOT(p1).

Example:
To set the position of a player at (3, 2) on board 0 and unset the box position at the same location, use the following code:
`clauses.append([encodeAtom(1, 3, 2, 0), -encodeAtom(0, 3, 2, 0)])`


In [95]:
def create_initial_clauses(clauses, StartBoard):

    height = len(StartBoard)
    width = len(StartBoard[0])

    for i in range(height):
        for j in range(width):
            if StartBoard[i][j] == 1 or StartBoard[i][j] == 0:
              clauses.append ([-encodeAtom(1, i, j, 0)])
              clauses.append ([-encodeAtom(0, i, j, 0)])

            elif StartBoard[i][j] == 'p':
              clauses.append ([encodeAtom(1, i, j, 0)])
              clauses.append ([-encodeAtom(0, i, j, 0)])

            elif StartBoard[i][j] == 'b':
              clauses.append ([-encodeAtom(1, i, j, 0)])
              clauses.append ([encodeAtom(0, i, j, 0)])

    return clauses

## 2. Final Board


For the first function, you must add a clause that satisfies our final board condition. For the second function, remove the added condition in case of unsatisfiability in the assumed steps.

In [96]:
def create_Final_clause(clauses, goal_position, max_steps):
  clauses.append([encodeAtom(0, goal_position[0] - 1, goal_position[1] - 1, max_steps)])  # goal position in one-based
  return clauses

def Delete_Final_clause(clauses, goal_position, max_steps):
  clauses.clauses.remove([encodeAtom(0, goal_position[0] - 1, goal_position[1] - 1, max_steps)])
  return clauses

## 3. Transition

In [97]:
def create_transition_clauses(clauses, StartBoard, step_number):
    height = len(StartBoard)
    width = len(StartBoard[0])

    for i in range (height):
        for j in range (width):
            # condition 2.1 --> it cant have both player and box (~(p and b))
            # it can cover NO BOX AND NO PLAYER
            clauses.append ([-encodeAtom(1, i, j, step_number), -encodeAtom(0, i, j, step_number)])


            if (StartBoard[i][j] == 1):
              clauses.append ([-encodeAtom(1, i, j, step_number)])
              clauses.append ([-encodeAtom(0, i, j, step_number)])
              continue  # if it is a wall, no other conditions are assigned

            # condition 2.3 -> at least one of the adj should have contained player in the prev move
            clauses_for_cell = []
            clauses_for_cell.append(-encodeAtom(1, i, j, step_number))
            if i - 1 >= 0:
              clauses_for_cell.append(encodeAtom(1, i - 1, j, step_number - 1))
            if j - 1 >= 0:
              clauses_for_cell.append(encodeAtom(1, i, j - 1, step_number - 1))
            if i + 1 < height:
              clauses_for_cell.append(encodeAtom(1, i + 1, j, step_number - 1))
            if j + 1 < width:
              clauses_for_cell.append(encodeAtom(1, i, j + 1, step_number - 1))
            clauses.append(clauses_for_cell)

            # to have exactly one player, we add the prev condtion with the difference of step_number
            # then we add a condition to have exact one player in each cell
            clauses_for_cell = []
            clauses_for_cell.append(-encodeAtom(1, i, j, step_number - 1))
            if i - 1 >= 0:
              clauses_for_cell.append(encodeAtom(1, i - 1, j, step_number))
            if j - 1 >= 0:
              clauses_for_cell.append(encodeAtom(1, i, j - 1, step_number))
            if i + 1 < height:
              clauses_for_cell.append(encodeAtom(1, i + 1, j, step_number))
            if j + 1 < width:
              clauses_for_cell.append(encodeAtom(1, i, j + 1, step_number))
            clauses.append(clauses_for_cell)

            # exactly one player in each cell ---> ~(P1 AND P2)
            for sec_i in range(height):
              for sec_j in range(width):
                if sec_i == i and sec_j == j:
                  continue
                clauses_for_cell = [-encodeAtom(1, i, j, step_number), -encodeAtom(1, sec_i, sec_j, step_number)]
                clauses.append(clauses_for_cell)
                clauses_for_cell = [-encodeAtom(0, i, j, step_number), -encodeAtom(0, sec_i, sec_j, step_number)]
                clauses.append(clauses_for_cell)


            # condition 2.4 --> if P(0, i, j, step_number) -> (P(0, i + change, j + change, step - 1) and P(1, i + change2, j + change2, step - 1) and P(1, i + change, j + change, step))
            # cover boxes. it was either in the last position, or it was after a player, and player has moved to the i,j
            direction = []
            if i - 1 >= 0 and i - 2 >= 0:
              this_direction = []
              this_direction.append(encodeAtom(1, i - 1, j, step_number))
              this_direction.append(encodeAtom(1, i - 2, j, step_number - 1))
              this_direction.append(encodeAtom(0, i - 1, j, step_number - 1))
              direction.append(this_direction)

            if j - 1 >= 0 and j - 2 >= 0:
              this_direction = []
              this_direction.append(encodeAtom(1, i, j - 1, step_number))
              this_direction.append(encodeAtom(1, i, j - 2, step_number - 1))
              this_direction.append(encodeAtom(0, i, j - 1, step_number - 1))
              direction.append(this_direction)

            if i + 1 < height and i + 2 < height:
              this_direction = []
              this_direction.append(encodeAtom(1, i + 1, j, step_number))
              this_direction.append(encodeAtom(1, i + 2, j, step_number - 1))
              this_direction.append(encodeAtom(0, i + 1, j, step_number - 1))
              direction.append(this_direction)

            if j + 1 < width and j + 2 < width:
              this_direction = []
              this_direction.append(encodeAtom(1, i, j + 1, step_number))
              this_direction.append(encodeAtom(1, i, j + 2, step_number - 1))
              this_direction.append(encodeAtom(0, i, j + 1, step_number - 1))
              direction.append(this_direction)

            if (len(direction) == 4):
              for one in direction[0]:
                for two in direction[1]:
                  for thr in direction[2]:
                    for four in direction[3]:
                      atom_condition = []
                      atom_condition.append(encodeAtom(0, i, j, step_number - 1))
                      atom_condition.append(-encodeAtom(0, i, j, step_number))
                      atom_condition.append(one)
                      atom_condition.append(two)
                      atom_condition.append(thr)
                      atom_condition.append(four)
                      clauses.append(atom_condition)

            elif (len(direction) == 3):
               for one in direction[0]:
                for two in direction[1]:
                  for thr in direction[2]:
                      atom_condition = []
                      atom_condition.append(encodeAtom(0, i, j, step_number - 1))
                      atom_condition.append(-encodeAtom(0, i, j, step_number))
                      atom_condition.append(one)
                      atom_condition.append(two)
                      atom_condition.append(thr)
                      clauses.append(atom_condition)

            elif (len(direction) == 2):
               for one in direction[0]:
                for two in direction[1]:
                      atom_condition = []
                      atom_condition.append(encodeAtom(0, i, j, step_number - 1))
                      atom_condition.append(-encodeAtom(0, i, j, step_number))
                      atom_condition.append(one)
                      atom_condition.append(two)
                      clauses.append(atom_condition)

            elif (len(direction) == 1):
               for one in direction[0]:
                      atom_condition.append(encodeAtom(0, i, j, step_number - 1))
                      atom_condition.append(-encodeAtom(0, i, j, step_number))
                      atom_condition.append(one)
                      clauses.append(atom_condition)


            # !!!! MY FAILED ATTEMPT TO WRITE THE EXACT THING WRITTEN IN GUIDE:

            # condition 2.2 --> if P(1, i, j, step - 1) -> exactly one
            # the normal form is m -> (p1 and ~p2 and ~p3 and ~p4) or (~p1 and p2 and ~p3 and ~p4)
            #                         (~p1 and ~p2 and p3 and ~p4) or (~p1 and ~p2 and ~p3 and p4)
            # so as CNF it is all other states in truth table with AND between
            # if exactly one of them is true (it means 1 in exactly one of them -> sum of them is 1) shouldn't be added

            # direction = []
            # for change in range (0, 4):
            #   this_direction = []
            #   this_direction.append(-encodeAtom(1, i + dx[change], j + dy[change], step_number))
            #   this_direction.append(encodeAtom(1, i + dx[change], j + dy[change], step_number))
            #   direction.append(this_direction)

            # if i - 1 >= 0:
            #   this_direction = []
            #   this_direction.append(-encodeAtom(1, i - 1, j, step_number))
            #   this_direction.append(encodeAtom(1, i - 1, j, step_number))
            #   direction.append(this_direction)

            # if j - 1 >= 0:
            #   this_direction = []
            #   this_direction.append(-encodeAtom(1, i, j - 1, step_number))
            #   this_direction.append(encodeAtom(1, i, j - 1, step_number))
            #   direction.append(this_direction)

            # if i + 1 < height:
            #   this_direction = []
            #   this_direction.append(-encodeAtom(1, i + 1, j, step_number))
            #   this_direction.append(encodeAtom(1, i + 1, j, step_number))
            #   direction.append(this_direction)

            # if j + 1 < width:
            #   this_direction = []
            #   this_direction.append(-encodeAtom(1, i, j + 1, step_number))
            #   this_direction.append(encodeAtom(1, i, j + 1, step_number))
            #   direction.append(this_direction)

            # if len (direction) == 4:
            #   for one in range(0, 2):
            #     for two in range (0, 2):
            #       for thr in range (0, 2):
            #         for four in range (0, 2):
            #           if one + two + thr + four == 3:
            #             continue
            #           atom_condition = []
            #           atom_condition.append (-encodeAtom(1, i, j, step_number - 1))
            #           atom_condition.append(direction[0][one])
            #           atom_condition.append(direction[1][two])
            #           atom_condition.append(direction[2][thr])
            #           atom_condition.append(direction[3][four])
            #           clauses.append(atom_condition)

            # elif len(direction) == 3:
            #   for one in range(0, 2):
            #     for two in range (0, 2):
            #       for thr in range (0, 2):
            #           if one + two + thr== 1:
            #             continue
            #           atom_condition = []
            #           atom_condition.append (-encodeAtom(1, i, j, step_number - 1))
            #           atom_condition.append(direction[0][one])
            #           atom_condition.append(direction[1][two])
            #           atom_condition.append(direction[2][thr])
            #           clauses.append(atom_condition)

            # elif len (direction) == 2:
            #   for one in range(0, 2):
            #     for two in range (0, 2):
            #           if one + two== 1:
            #             continue
            #           atom_condition = []
            #           atom_condition.append (-encodeAtom(1, i, j, step_number - 1))
            #           atom_condition.append(direction[0][one])
            #           atom_condition.append(direction[1][two])
            #           clauses.append(atom_condition)

            # elif len (direction) == 1:
            #   for one in range(0, 2):
            #           if one == 1:
            #             continue
            #           atom_condition = []
            #           atom_condition.append (-encodeAtom(1, i, j, step_number - 1))
            #           atom_condition.append(direction[0][one])
            #           clauses.append(atom_condition)



            # !!! BUG ALERT: THE CHECK_WALL FUNCTION WENT WRONG, SO I WAS SUPPOSED TO WRITE ALL THE STATES: !!!

            # # condition 2.3 -->  if P(1, i, j, step_number) -> P(1, i + change, j + change, step_number - 1)
            # # if it is  m -> (p1 or p2 or p3 or p4)
            # # as a CNF it is:   (~m or p1 or p2 or p3 or p4)
            # # Pi represents player in one of i directions 1 <= i <= 4

            # # for one in range(0, 2):
            # #   for two in range (0, 2):
            # #     for thr in range (0, 2):
            # #       for four in range (0, 2):
            # #         if (one + two + thr + four == 1):
            # #           continue
            # #         atom_condition = []
            # #         atom_condition.append (-encodeAtom(1, i, j, step_number - 1))

            # #         if check_wall(StartBoard, decodeAtom(direction[0][one]), height, width):
            # #           atom_condition.append(direction[0][one])
            # #         if check_wall(StartBoard, decodeAtom(direction[1][two]), height, width):
            # #           atom_condition.append(direction[1][two])
            # #         if check_wall(StartBoard, decodeAtom(direction[2][thr]), height, width):
            # #           atom_condition.append(direction[2][thr])
            # #         if check_wall(StartBoard, decodeAtom(direction[3][four]), height, width):
            # #           atom_condition.append(direction[3][four])

            # #         clauses.append(atom_condition)

    return clauses

# Decode Solution

In [98]:
def print_solution(model, StartBoard, number_of_steps):

    print('\n## Solution:')
    for t in range(number_of_steps + 1):
        print("\n STEP " + str(t) + ' Board:')

        # deepCopy will copy Board in 2D array (pass by value)
        A = copy.deepcopy(StartBoard)
        height = len(StartBoard)
        width = len(StartBoard[0])
        for i in range (height):
          for j in range (width):
            if A[i][j] != 1:
              A[i][j] = 0  # initialize

        for atom in model:
          decoded_atom = decodeAtom(atom) # it returns a tuple of (kind, i, j ,t)
          if decoded_atom[3] == t: # it means it is related to this step
            if decoded_atom[0] == 1: # it means that player was not here
                  if atom > 0:
                      A[decoded_atom[1]][decoded_atom[2]] = 'p'
            else:
                if atom > 0:
                  A[decoded_atom[1]][decoded_atom[2]] = 'b'

        for a in A:
          print (a)

    print('\n## END of solution')
    return True

# Solver function

In [99]:

def solve_sokoban(StartBoard, goal_position, max_steps):

    clauses = CNF()
    clauses = create_initial_clauses(clauses, StartBoard)

    number_of_steps = 0

    for i in range (1, max_steps + 1):   # max_step + 1 --> so it will check max-step, too
      create_transition_clauses(clauses, StartBoard, i)
      create_Final_clause (clauses, goal_position, i)
      solver = Solver(bootstrap_with = clauses)
      if solver.solve():
        model = solver.get_model()
        print_solution(model, StartBoard, i)
        solver.delete()
        Delete_Final_clause(clauses, goal_position, i)
        return
      else:
        Delete_Final_clause(clauses, goal_position, i)

    return None

#Game Creator
creat games here and run it.

In [100]:
if __name__ == '__main__':

    goal_position = (3, 4)
    StartBoard = [
        [0, 0, 0, 0, 0],
        ['p', 'b', 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0]
    ]

    max_steps = 11

    solve_sokoban( StartBoard, goal_position, max_steps)


## Solution:

 STEP 0 Board:
[0, 0, 0, 0, 0]
['p', 'b', 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 1 Board:
['p', 0, 0, 0, 0]
[0, 'b', 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 2 Board:
[0, 'p', 0, 0, 0]
[0, 'b', 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 3 Board:
[0, 0, 0, 0, 0]
[0, 'p', 1, 0, 0]
[0, 'b', 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 4 Board:
[0, 0, 0, 0, 0]
['p', 0, 1, 0, 0]
[0, 'b', 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 5 Board:
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
['p', 'b', 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 6 Board:
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 'p', 'b', 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

 STEP 7 Board:
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 'p', 'b', 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]

## END of solution
